# Objective:
This notebook implements a Generative Adversarial Network (GAN) to generate
realistic human face images. The generator creates new images from random noise,
while the discriminator learns to distinguish between real and fake faces.

##Dataset: CelebA ( [CelebFaces Attributes](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset)) Dataset

## Tasks Covered:
1. Preprocess face images (resize to 64x64 and normalize pixel values).
2. Implement GAN architecture (Generator + Discriminator using CNN layers).
3. Train the GAN using Binary Cross-Entropy loss.
4. Generate and visualize new face images after training.



---



# Setup and decompression from Drive

In [ ]:
# Setup and Unzip from Drive
import os, glob, zipfile
from pathlib import Path

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Path to your ZIP on Drive
ZIP_PATH = "/content/drive/MyDrive/Dataset/CelebA.zip"
EXTRACT_PATH = Path("/content/celeba_temp")

if not Path(ZIP_PATH).exists():
    raise FileNotFoundError(f"file not found {ZIP_PATH}")

# Unzip once to /content
EXTRACT_PATH.mkdir(parents=True, exist_ok=True)
print("Unzip from the drive", EXTRACT_PATH)
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    zf.extractall(EXTRACT_PATH)

# Locate images folder even if nested img_align_celeba/img_align_celeba
def find_img_dir(root: Path):
    for p in root.rglob("img_align_celeba"):
        if any((Path(p)).glob("*.jpg")):
            return p
    return None

imgs_dir = find_img_dir(EXTRACT_PATH)
if imgs_dir is None:
    raise RuntimeError("img_align_celeba was not found inside the source file.")
print("Photo file", imgs_dir)



---



# Data Loader

Converts images to 64x64 and values ​​[0,1]

In [ ]:
# Dataset and DataLoader
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.files = sorted(glob.glob(str(self.root_dir / "*.jpg")))
        if len(self.files) == 0:
            raise RuntimeError("There are no .jpg images inside the specified folder.")
        self.transform = transform or transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ConvertImageDtype(torch.float32), # Maintains [0,1]
        ])

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        img = torchvision.io.read_image(self.files[idx]).float() / 255.0
        if img.shape[0] == 1:  #Make sure its 3 channels
            img = img.repeat(3, 1, 1)
        img = self.transform(img)
        return img, 0

batch_size = 128 if torch.cuda.is_available() else 32
num_workers = 2 if torch.cuda.is_available() else 0

dataset = CelebADataset(imgs_dir)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True,
                    num_workers=num_workers, pin_memory=torch.cuda.is_available(),
                    drop_last=True)

print("Number of images:", len(dataset), " | Batches per epoch:", len(loader))


---



# Models (Generator & Discriminator)

DCGAN with sigmaoid output [0,1]

In [ ]:
# Generator and Discriminator
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100, g_base=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, g_base*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(g_base*8), nn.ReLU(True),
            nn.ConvTranspose2d(g_base*8, g_base*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(g_base*4), nn.ReLU(True),
            nn.ConvTranspose2d(g_base*4, g_base*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(g_base*2), nn.ReLU(True),
            nn.ConvTranspose2d(g_base*2, g_base, 4, 2, 1, bias=False),
            nn.BatchNorm2d(g_base), nn.ReLU(True),
            nn.ConvTranspose2d(g_base, 3, 4, 2, 1, bias=False),
            nn.Sigmoid() #Output [0,1]
        )
    def forward(self, z): return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, d_base=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, d_base, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d_base, d_base*2, 4, 2, 1, bias=False), nn.BatchNorm2d(d_base*2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d_base*2, d_base*4, 4, 2, 1, bias=False), nn.BatchNorm2d(d_base*4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d_base*4, d_base*8, 4, 2, 1, bias=False), nn.BatchNorm2d(d_base*8), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d_base*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # Output probability
        )
    def forward(self, x): return self.net(x).view(-1)

# Preparing training models and details
z_dim = 100
G, D = Generator(z_dim).to(device), Discriminator().to(device)
optG = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
optD = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))
criterion = nn.BCELoss()

from torchvision.utils import save_image
out_dir = Path("/content/gan_outputs")
(out_dir / "samples").mkdir(parents=True, exist_ok=True)
(out_dir / "ckpts").mkdir(parents=True, exist_ok=True)

epochs = 15
fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)
G_losses, D_losses = [], []



---



# Training + Sample Savings + Checkpoints

Alternating updates, saving ckpt files and samples for each era

In [ ]:
# Training Loop (with checkpoints & samples)
import math
from tqdm.auto import tqdm

print("Start training . . . ")
for epoch in range(1, epochs+1):
    running_D, running_G = 0.0, 0.0

    # minimal change: wrap the loader with tqdm
    for i, (real_imgs, _) in enumerate(tqdm(loader, desc=f"Epoch {epoch}/{epochs}", dynamic_ncols=True), start=1):  # NEW
        real_imgs = real_imgs.to(device)
        bsz = real_imgs.size(0)
        real_labels = torch.ones(bsz, device=device)
        fake_labels = torch.zeros(bsz, device=device)

        # Train D
        optD.zero_grad()
        loss_real = criterion(D(real_imgs), real_labels)
        z = torch.randn(bsz, z_dim, 1, 1, device=device)
        fake = G(z).detach()
        loss_fake = criterion(D(fake), fake_labels)
        loss_D = loss_real + loss_fake
        loss_D.backward(); optD.step()

        # Train G
        optG.zero_grad()
        z = torch.randn(bsz, z_dim, 1, 1, device=device)
        gen = G(z)
        loss_G = criterion(D(gen), real_labels)
        loss_G.backward(); optG.step()

        G_losses.append(loss_G.item()); D_losses.append(loss_D.item())

        # brief live stats in the bar
        running_D += loss_D.item()
        running_G += loss_G.item()
        if i % 10 == 0:  # update the bar every 10 steps to keep it light
            avgD = running_D / i
            avgG = running_G / i
            tqdm.write(f"[Epoch {epoch}/{epochs}] step {i}/{len(loader)}  D={loss_D.item():.4f}  G={loss_G.item():.4f}  avgD={avgD:.4f}  avgG={avgG:.4f}")  # NEW

    # Preserving samples after each epoch
    with torch.no_grad():
        sample = G(fixed_noise).cpu()
    save_image(sample, str(out_dir / "samples" / f"epoch_{epoch:03d}.png"),
               nrow=int(math.sqrt(fixed_noise.size(0))))

    torch.save(
        {"G": G.state_dict(), "D": D.state_dict(),
         "optG": optG.state_dict(), "optD": optD.state_dict(),
         "epoch": epoch},
        str(out_dir / "ckpts" / f"ckpt_{epoch:03d}.pt")
    )

    print(f"Epoch {epoch}/{epochs} done")



---



# Loss curve + visual comparison (real vs. generated)

Visual evaluation

In [ ]:
#Curves + Visual Comparison
import matplotlib.pyplot as plt
from torchvision.utils import save_image

# Loss Curve
plt.figure()
plt.plot(D_losses, label="D"); plt.plot(G_losses, label="G")
plt.legend(); plt.title("GAN Training Loss"); plt.tight_layout()
plt.savefig(str(out_dir / "loss_curves.png")); plt.close()

#Real samples against a generator
real_batch, _ = next(iter(loader))
n = min(32, real_batch.size(0), 64)
with torch.no_grad():
    fake_batch = G(torch.randn(n, z_dim, 1, 1, device=device)).cpu()

comp = torch.cat([real_batch[:n].cpu(), fake_batch[:n]], dim=0)
save_image(comp, str(out_dir / "samples" / "compare_final.png"), nrow=max(1, n//2))

print("save inside", out_dir)
print(" - loss_curves.png")
print(" - samples/epoch_XXX.png")
print(" - samples/compare_final.png")
print(" - ckpts/ckpt_XXX.pt")

# Show example
from IPython.display import Image, display
display(Image(filename=str(out_dir / "samples" / f"epoch_{epochs:03d}.png")))
display(Image(filename=str(out_dir / "samples" / "compare_final.png")))



---



# Analysis results

### **GAN Results Analysis (CelebA Dataset)**

**Experiment Description:**  
A DCGAN model was trained on the elebA dataset, which contains over 200,000 human face images.  
The experiment was executed in Google Colab (T4 GPU) for 15 epochs.  
The goal was to teach the generator to learn the statistical distribution of real faces in order to create new, realistic ones from random noise.

---

### **Loss Curve Analysis**  
- The **Discriminator (D)** loss gradually decreased and stabilized around 0.3, indicating that it learned effectively without overpowering the generator.  
- The **Generator (G)** loss fluctuated between 3 – 5, which is expected and shows a healthy adversarial balance between both networks.  
- No signs of mode collapse appeared, as the generator continued producing diverse samples.

---

### **Image Quality Evolution**  
- **Early epochs (1–3):** The generated images were blurry and noisy, as the generator was still learning basic facial structures.  
- **Middle epochs (7–10):** Facial features started to appear — eyes, hair, and skin tones became more coherent.  
- **Final epochs (15):** The model produced mostly realistic faces with consistent shapes and lighting.  

---

### **General Conclusion**  
The DCGAN successfully **learned the facial data distribution** and was able to generate new, realistic-looking faces.  
Both generator and discriminator losses remained stable, confirming balanced adversarial training.  

---

### **Suggestions for Future Improvement**  
- Increase the training epochs to 30–50 for higher visual detail.  
- Normalize the dataset to [-1, +1] instead of dividing by 255 to improve stability.  
- Try more advanced architectures such as WGAN-GP or StyleGAN for enhanced realism and stability.